Part 1: Using Jax to compute gradients

TODO: saddle point example showing static point jacobian and hessian instabilit

In [1]:
import jax
import jax.numpy as jnp

In [2]:
def f(x):
    x1, x2 = x
    return jnp.power(x1, 2)  + jnp.power(x1, 3)

In [3]:
x= jnp.array([1.0, -1.0])
jax.jacrev(f)(x)

Array([4., 3.], dtype=float32)